In [1]:
import os
import numpy as np
from tqdm import tqdm

from transformers import (
    get_scheduler,
    pipeline,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    AdamW,
)
from datasets import load_dataset
import evaluate

import torch
from torch.utils.data import DataLoader

In [2]:
torch.cuda.is_available()

True

## Data Acquisition and Analysis

In [3]:
# https://huggingface.co/datasets/GonzaloA/fake_news
dataset = load_dataset("GonzaloA/fake_news")
dataset

Using custom data configuration GonzaloA--fake_news-1fe2b42e1fa111c8
Found cached dataset parquet (C:/Users/Josu Alonso/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-1fe2b42e1fa111c8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})

In [4]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/9 [00:00<?, ?ba/s]

Loading cached processed dataset at C:\Users\Josu Alonso\.cache\huggingface\datasets\GonzaloA___parquet\GonzaloA--fake_news-1fe2b42e1fa111c8\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-465f8208a4e3cf5a.arrow
Loading cached processed dataset at C:\Users\Josu Alonso\.cache\huggingface\datasets\GonzaloA___parquet\GonzaloA--fake_news-1fe2b42e1fa111c8\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-282d5055c57d298e.arrow


In [6]:
tokenized_dataset = tokenized_dataset.remove_columns(["Unnamed: 0", "title", "text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset["train"].column_names

['labels', 'input_ids', 'attention_mask']

## Model Training

In [7]:
train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size=32, collate_fn=data_collator
)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [9]:
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

c:\Users\Josu Alonso\Projects\fake-news-detector\env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [11]:
total_steps = len(train_dataloader) * num_epochs
progress_bar = tqdm(range(num_training_steps))

model.train()
with tqdm(total=total_steps) as pbar:
    for epoch in range(num_epochs):
        for i, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            pbar.write(f'processed: {i * (epoch + 1) / total_steps * 100:.2f}%')
            pbar.update(1)

  0%|          | 0/2286 [00:00<?, ?it/s]You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                        
  0%|          | 0/2286 [00:01<?, ?it/s]

processed: 0.00%


                                        
  0%|          | 0/2286 [00:01<?, ?it/s]          

processed: 0.04%


                                        
  0%|          | 0/2286 [00:02<?, ?it/s]        

...

processed: 99.48%


                                        
  0%|          | 0/2286 [09:03<?, ?it/s]           

processed: 99.61%


                                        
                                                   
100%|██████████| 2286/2286 [09:03<00:00,  4.20it/s]

processed: 99.74%
processed: 99.87%


In [12]:
model.save_pretrained(os.path.join("models", "fake-news-detector"))

## Model Evaluation

In [19]:
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score
)

In [24]:
model.eval()

predictions = []

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    running_predictions = torch.argmax(logits, dim=-1).to('cpu').numpy().tolist()
    predictions.extend(running_predictions)

In [27]:
print(classification_report(tokenized_dataset["validation"]["labels"], predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3723
           1       0.99      0.99      0.99      4394

    accuracy                           0.99      8117
   macro avg       0.99      0.99      0.99      8117
weighted avg       0.99      0.99      0.99      8117



In [28]:
print(confusion_matrix(tokenized_dataset["validation"]["labels"], predictions))

[[3686   37]
 [  53 4341]]
